---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [11]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [30]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    # read university_town data        
    #with open("university_towns.txt","r") as f:
         #d = f.readlines()  # remove '\n' from list 
    #d = pd.DataFrame(d)
    d = pd.read_fwf(r"university_towns.txt", header=None)
    d.columns = ['RegionName']
    
    state_dict = [i for i in d['RegionName'].tolist() if '[edit]' in i]
    d['State_Index'] = list(map(lambda x: x in state_dict, d['RegionName'].tolist()))
    # map is similar to apply, lambda is writting a function 
    d['State'] = np.where(d['State_Index'] == True, d['RegionName'], None)
    d['State'] = d['State'].fillna(method='ffill')
    d = d[d['State_Index'] == False]
    
    d['RegionName'] = d['RegionName'].str.replace(r"\[.*\]","")
    #d['RegionName'] = d['RegionName'].str.replace(r" \(.*\)","")
    d['State'] = d['State'].str.replace(r"\[.*\]","")
    d["RegionName"] = d["RegionName"].map(lambda x:x.split("(")[0].strip())
    
    out = d[['State','RegionName']].reset_index(drop = True)
    
    return out
get_list_of_university_towns()

In [2]:
def GDP():
    gdp = pd.read_excel('gdplev.xls', header = 4, usecols = [4,5,6])
    gdp = gdp.drop(gdp.index[[0,1]])
    gdp.columns = ['Quarter', 'GDP of current dollars', 'GDP of 2009 dollars']
    gdp = gdp [gdp['Quarter'] >= '2000q1'].reset_index(drop = True)
    
    gdp['Prev_GDP'] = gdp['GDP of 2009 dollars'].shift(1)
    gdp['GDP_Diff'] = gdp['GDP of 2009 dollars'] - gdp['Prev_GDP']
    gdp['Next_Year_GDPDif'] = gdp['GDP_Diff'].shift(-1)
    return gdp

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = GDP()
    recession = gdp[(gdp['GDP_Diff']<0) & (gdp['Next_Year_GDPDif']<0)].reset_index(drop = True)
    return recession['Quarter'][0]

get_recession_start()

,Quarter,GDP of current dollars,GDP of 2009 dollars
2,1947q1,243.1,1934.5
3,1947q2,246.3,1932.3
4,1947q3,250.1,1930.3
5,1947q4,260.3,1960.7
6,1948q1,266.2,1989.5
7,1948q2,272.9,2021.9
8,1948q3,279.5,2033.2
9,1948q4,280.7,2035.3
10,1949q1,275.4,2007.5
11,1949q2,271.7,2000.8


In [16]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = GDP()
    gdp = gdp [gdp['Quarter'] >= '2008q3']
    recession_end = gdp[(gdp['GDP_Diff']>0) & (gdp['Next_Year_GDPDif']>0)].reset_index(drop = True)
    return recession_end['Quarter'][1]

get_recession_end()

'2009q4'

In [17]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp = GDP()
    recession = gdp[(gdp['Quarter'] >= '2008q3') & (gdp['Quarter'] <= '2009q3')].reset_index(drop = True)
    bottom = recession[recession['GDP of 2009 dollars'] ==  np.min(recession['GDP of 2009 dollars'])]
    return bottom['Quarter'].tolist()[0]

get_recession_bottom()

'2009q2'

In [13]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    house = pd.read_csv("City_Zhvi_AllHomes.csv")
    info = house.iloc[:,[2,1]]
    df = house.iloc[:,6:]
    #convert monthly data to quarterly, and calculate the mean of quarterly data 
    df = df.groupby(pd.PeriodIndex(df,freq='Q'),axis=1).mean().rename(columns=lambda c: str(c).lower())
    #find the position of '2001q1' and get columns afterwards
    df = df.iloc[:,np.where(df.columns == '2000q1')[0][0]:]
    out = pd.merge(info, df, left_index=True, right_index=True, how='outer')
    out['State'] = out['State'].map(states)
    out = out.set_index(['State','RegionName'])
    return out

convert_housing_data_to_quarters()

202266.66666666666

In [63]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    housing_data = convert_housing_data_to_quarters()

    start_index = np.where(housing_data.columns == get_recession_start())[0][0] 
    bottom_index = np.where(housing_data.columns == get_recession_bottom())[0][0]

    housing_data = housing_data.iloc[:,[start_index-1, bottom_index]]
    housing_data['Ratio'] = housing_data['2008q2'].div(housing_data['2009q2'])
    #find  month before start of recession & month of recession bottom
    recession = housing_data['Ratio'].reset_index()

    university = get_list_of_university_towns()
    merge1 = pd.merge(university, recession, how = 'inner', on = ['State','RegionName'])
    merge1['univ'] = 'yes'  # find the university city 
    merge2 = pd.merge(merge1, recession, how = 'outer', on = ['State','RegionName','Ratio'])
    merge2['univ'] = merge2['univ'].fillna('no')  # find the non-university city 
    merge2 = merge2.drop_duplicates(keep = False)

    ratio1 = merge2.loc[merge2['univ']=='yes'].Ratio
    ratio2 = merge2.loc[merge2['univ']=='no'].Ratio

    # t-test 
    t,p_value = ttest_ind(ratio1,ratio2,nan_policy = 'omit')
    different =  True if p_value < 0.01 else False
    better = 'university town' if ratio1.mean() < ratio2.mean() else 'non-university town'
    out = (different, p_value, better)
    
    return out

run_ttest()

(True, 0.0027240637047531249, 'university towns')